# SF3D API Server - JupyterHub (v3 - Real SF3D)

This notebook runs a FastAPI server using **actual Stable Fast 3D** from Stability AI.

## ⚠️ IMPORTANT: Run cells in order!

**Setup Steps:**
1. Run Cell 1: Install dependencies
2. **RESTART KERNEL** (Kernel → Restart)
3. Run Cell 2: Clone SF3D repository
4. Run Cell 3: Import libraries and load model
5. Run Cell 4: Define API endpoints
6. Run Cell 5: Start server (keep running)
7. Test from Mac: `python tests/sf3d_api_client.py <image_path>`

## Cell 1: Install Dependencies

**Run this first, then RESTART KERNEL!**

In [ ]:
%%bash
# Install PyTorch with CUDA 11.8 (matching server)
pip install --user torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install FastAPI and web server
pip install --user fastapi uvicorn[standard] python-multipart

# Install core dependencies
pip install --user pillow numpy omegaconf einops

# Install 3D libraries
pip install --user trimesh[easy] pymeshlab

# Install ML libraries
pip install --user transformers accelerate safetensors huggingface-hub

# Install additional deps for SF3D
pip install --user opencv-python imageio rembg

echo ""
echo "✅ Installation complete!"
echo ""
echo "⚠️  IMPORTANT: Now go to Kernel → Restart Kernel"
echo "   Then run Cell 2 to clone SF3D repository."

## Cell 2: Clone SF3D Repository

**After restarting kernel, run this cell**

In [ ]:
import os
import sys
from pathlib import Path

# Check if SF3D already cloned
sf3d_dir = Path.home() / "stable-fast-3d"

if sf3d_dir.exists():
    print(f"✅ SF3D already cloned at: {sf3d_dir}")
else:
    print("📥 Cloning Stable Fast 3D repository...")
    !git clone https://github.com/Stability-AI/stable-fast-3d.git ~/stable-fast-3d
    print(f"✅ SF3D cloned to: {sf3d_dir}")

# Add to Python path
sys.path.insert(0, str(sf3d_dir))
print(f"\n✅ Added to Python path: {sf3d_dir}")

# Verify it exists
if (sf3d_dir / "sf3d").exists():
    print("✅ SF3D package found")
else:
    print("❌ SF3D package not found - check repository structure")
    print(f"   Contents: {list(sf3d_dir.iterdir())}")

## Cell 3: Load SF3D Model

**This loads the actual SF3D model (first time: downloads ~2GB)**

In [ ]:
import time
import torch
import numpy as np
from PIL import Image

print("Importing SF3D...")
try:
    # Try importing from the cloned repository
    from sf3d.system import SF3D
    print("✅ SF3D imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\nTrying alternative import method...")
    
    # Add subdirectories to path
    import sys
    sf3d_dir = Path.home() / "stable-fast-3d"
    sys.path.insert(0, str(sf3d_dir / "sf3d"))
    
    from system import SF3D
    print("✅ SF3D imported (alternative method)")

# Configure device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {device}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  WARNING: CUDA not available! Will use CPU (very slow)")

# Load SF3D model
print("\nLoading SF3D model from Hugging Face...")
print("(First time: downloads ~2GB model, may take 30-60 seconds)")
start_time = time.time()

try:
    model = SF3D.from_pretrained(
        "stabilityai/stable-fast-3d",
        config_name="config.yaml",
        weight_name="model.safetensors",
    )
    model = model.to(device)
    model.eval()
    
    print(f"\n✅ SF3D model loaded in {time.time() - start_time:.1f}s")
    print("   Model: stabilityai/stable-fast-3d")
    
except Exception as e:
    print(f"\n❌ Error loading model: {e}")
    print("\nTroubleshooting:")
    print("  1. Check internet connection")
    print("  2. May need Hugging Face token for some models")
    print("  3. Check Hugging Face Hub status")
    raise

# Create output directory
output_dir = Path("/tmp/sf3d_outputs")
output_dir.mkdir(exist_ok=True)
print(f"\nOutput directory: {output_dir}")
print("\n✅ Ready to generate meshes with SF3D!")

## Cell 4: Define API Endpoints

In [ ]:
import io
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# Initialize FastAPI
app = FastAPI(
    title="Stable Fast 3D API",
    description="Generate high-quality 3D meshes from images using SF3D",
    version="3.0.0"
)

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        "message": "Stable Fast 3D API Server",
        "status": "running",
        "device": device,
        "model": "stabilityai/stable-fast-3d",
        "endpoints": {
            "/generate": "POST - Generate 3D mesh from image",
            "/health": "GET - Health check"
        }
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "device": device,
        "cuda_available": torch.cuda.is_available(),
        "model_loaded": model is not None,
        "model_type": "SF3D"
    }

@app.post("/generate")
async def generate_mesh(
    file: UploadFile = File(...),
    texture_resolution: int = Form(1024),
    remesh_option: str = Form("none"),
    foreground_ratio: float = Form(0.85)
):
    """
    Generate 3D mesh from uploaded image using SF3D.
    
    Parameters:
    - file: Image file (PNG, JPG)
    - texture_resolution: 512, 1024, or 2048 (default: 1024)
    - remesh_option: 'none', 'triangle', or 'quad' (default: 'none')
    - foreground_ratio: 0.5-1.0 (default: 0.85)
    
    Returns:
    - GLB file with UV-mapped textures
    """
    try:
        # Read and validate image
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))
        
        # Convert to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"\n[{time.strftime('%H:%M:%S')}] Received: {image.size}, texture={texture_resolution}, remesh={remesh_option}")
        
        # Generate mesh with SF3D
        start_time = time.time()
        
        with torch.no_grad():
            # SF3D run method
            output = model.run(
                image,
                bake_resolution=texture_resolution,
                remesh=remesh_option if remesh_option != 'none' else None,
                vertex_count=-1,  # Auto
            )
        
        generation_time = time.time() - start_time
        print(f"[{time.strftime('%H:%M:%S')}] Generated in {generation_time:.2f}s")
        
        # Save mesh
        timestamp = int(time.time() * 1000)
        output_path = output_dir / f"mesh_{timestamp}.glb"
        
        # SF3D output is a dict with 'mesh' key
        if isinstance(output, dict) and 'mesh' in output:
            mesh = output['mesh']
        else:
            mesh = output
        
        # Export as GLB
        mesh.export(str(output_path))
        
        file_size = output_path.stat().st_size
        print(f"[{time.strftime('%H:%M:%S')}] Saved: {output_path.name} ({file_size / 1024:.1f} KB)\n")
        
        # Return file
        return FileResponse(
            path=output_path,
            media_type="model/gltf-binary",
            filename=f"mesh_{timestamp}.glb",
            headers={
                "X-Generation-Time": str(generation_time),
                "X-File-Size": str(file_size),
                "X-Model": "SF3D"
            }
        )
    
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))

print("\n✅ API endpoints defined")
print("   Ready to start server!")

## Cell 5: Start Server

**⚠️ Keep this cell running!**

In [ ]:
# Configure server
PORT = 8765  # Change if port is in use
HOST = "0.0.0.0"

print("="*70)
print("🚀 Starting Stable Fast 3D API Server")
print("="*70)
print(f"Server URL: http://itp-ml.itp.tsoa.nyu.edu:{PORT}/")
print(f"Device: {device}")
print(f"Model: SF3D (stabilityai/stable-fast-3d)")
print(f"")
print("Features:")
print("  ✅ UV-mapped textures")
print("  ✅ High-quality meshes")
print("  ✅ Fast GPU inference (0.5-2s)")
print(f"")
print("Endpoints:")
print(f"  GET  http://itp-ml.itp.tsoa.nyu.edu:{PORT}/health")
print(f"  POST http://itp-ml.itp.tsoa.nyu.edu:{PORT}/generate")
print(f"")
print("Test from your Mac:")
print(f"  python tests/sf3d_api_client.py <image> --server http://itp-ml.itp.tsoa.nyu.edu:{PORT}")
print("="*70)
print("")
print("⚠️  KEEP THIS CELL RUNNING")
print("")

# Run server
try:
    uvicorn.run(app, host=HOST, port=PORT, log_level="info")
except KeyboardInterrupt:
    print("\n\n✅ Server stopped")
except Exception as e:
    print(f"\n\n❌ Server error: {e}")
    print("\nTroubleshooting:")
    print("  - Port in use? Change PORT above and re-run")
    print("  - Permission denied? Try PORT > 1024")